In [1]:
import os

from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from tempfile import gettempdir

from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDatasetVectorized
from l5kit.planning.vectorized.closed_loop_model import VectorizedUnrollModel
from l5kit.planning.vectorized.open_loop_model import VectorizedModel
from l5kit.vectorization.vectorizer_builder import build_vectorizer

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [2]:
#@title Download L5 Sample Dataset and install L5Kit
import os
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !wget https://raw.githubusercontent.com/lyft/l5kit/master/examples/setup_notebook_colab.sh -q
    !sh ./setup_notebook_colab.sh
    os.environ["L5KIT_DATA_FOLDER"] = open("./dataset_dir.txt", "r").read().strip()
else:
    print("Not running in Google Colab.")
    os.environ["L5KIT_DATA_FOLDER"] = "/home/chenyx/repos/l5kit/prediction-dataset"

Not running in Google Colab.


In [4]:
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./config.yaml")

In [5]:
# ===== INIT DATASET
train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()

vectorizer = build_vectorizer(cfg, dm)
train_dataset = EgoDatasetVectorized(cfg, train_zarr, vectorizer)
print(train_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


# Choosing the model

You can use this notebook to train not only Urban Driver, but also several ablations included in the paper. We provide the following options:
- Urban Driver: this is the default value and the model presented in the paper;
- Urban Driver without BPTT: this is an ablation of Urban Driver where we detach the gradient between steps;
- Open Loop Planner: this is vectorized model trained with simple behavioural cloning;

We now detail which config changes are necessary to obtain the baseline models - note that these are also required when loading pre-trained models for evaluation:

| Model     | Changes to config |
| ----------- | ----------- |
| Open Loop Planner  (BC-perturb)   | - history_num_frames_ego: 0 |
| Open Loop Planner with Ego History  (BC-perturb) | None  |
| Urban Driver without BPTT (MS Prediction)   | - future_num_frames: 32 <br/> - warmup_num_frames: 20|
| Urban Driver without BPTT (Ours)   | - future_num_frames: 32 <br/> - warmup_num_frames: 20 <br/> - detach_unroll: False|

In [5]:
URBAN_DRIVER = "Urban Driver"
OPEN_LOOP_PLANNER = "Open Loop Planner"

In [10]:
from tbsim.algos.l5kit_algos import L5TrafficModel, L5TransformerTrafficModel
from tbsim.configs import (
    ExperimentConfig,
    L5KitEnvConfig,
    L5KitTrainConfig,
    L5RasterizedPlanningConfig,
    L5KitVectorizedEnvConfig,
    L5TransformerPredConfig,
)
from tbsim.models.Transformer_model import Transformer_model
algo_config = L5TransformerPredConfig()
model = Transformer_model(algo_config)

# Prepare for training
Our `EgoDatasetVectorized` inherits from PyTorch `Dataset`; so we can use it inside a `Dataloader` to enable multi-processing.

In [8]:
train_cfg = cfg["train_data_loader"]
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"],
                              num_workers=train_cfg["num_workers"])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [11]:
tr_it = iter(train_dataloader)
data = next(tr_it)
model.forward(data)


AttributeError: 'Transformer_model' object has no attribute 'dyn'

In [84]:
data["timestamp"]




tensor([1581110868902077574, 1582059611302578516, 1583176950802484426,
        1571844370502093254, 1581462598202736866, 1576774713001940694,
        1579628327602012726, 1583276075102811396, 1583774647402133386,
        1583276584402449166, 1573491789402369066, 1579729003102116926])

# Training loop
Here, we purposely include a barebone training loop. Clearly, many more components can be added to enrich logging and improve performance. Still, the sheer size of our dataset ensures that a reasonable performance can be obtained even with this simple loop.

In [1]:
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []
model.train()
torch.set_grad_enabled(True)

for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    # Forward pass
    data = {k: v.to(device) for k, v in data.items()}
    result = model(data)
    loss = result["loss"]
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

NameError: name 'train_dataloader' is not defined

### Plot the train loss curve
We can plot the train loss against the iterations (batch-wise) to check if our model has converged.

In [ ]:
plt.plot(np.arange(len(losses_train)), losses_train, label="train loss")
plt.legend()
plt.show()

# Store the model

Let's store the model as a torchscript. This format allows us to re-load the model and weights without requiring the class definition later.

**Take note of the path, you will use it later to evaluate your planning model!**

In [ ]:
to_save = torch.jit.script(model.cpu())
path_to_save = f"{gettempdir()}/urban_driver.pt"
to_save.save(path_to_save)
print(f"MODEL STORED at {path_to_save}")

# Congratulations in training your very own Urban Driver model!
### What's Next

Now that your model is trained and safely stored, you can evaluate how it performs in our simulation:


### [Closed-loop evaluation](./closed_loop_test.ipynb)
In this setting the model **is in full control of the AV's** future movements.

## Pre-trained models
We provide a collection of pre-trained models, including both our proposed method and several ablations from our paper:
- [Urban Driver](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/urban_driver/BPTT.pt);
- [Urban Driver without BPTT](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/urban_driver/MS.pt);
- [Open Loop](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/urban_driver/OL.pt);
- [Open Loop with history](https://lyft-l5-datasets-public.s3-us-west-2.amazonaws.com/models/urban_driver/OL_HS.pt);

To use one of the models simply download the corresponding `.pt` file and load it in the evaluation notebooks - further, please see the comments above regarding necessary config changes.